In [2]:
import random
import time
import numpy as np
import tensorflow as tf

In [3]:
def load_data(path):
    print('Load ' + path)
    
    data = np.load(path)

    labels = list()
    titles = dict()
    embeddings = list()
    context = list()
    for data_dict in data:

            doc_id = data_dict['doc_index']
            titles[doc_id] = data_dict['doc_title']
            ctx = data_dict['doc_window']

            e = data_dict['pivot_embeddings']

            for emb in e:
                labels.append(doc_id)
                embeddings.append(emb)
                
                neg_ctx = negative_samples(data, ctx, 1)
                context.append(neg_ctx)

    return labels, embeddings, context, titles


def negative_samples(data, doc_context, num):
    negative_samples = []
    
    while len(negative_samples) < num:
        sample = random.choice(data)
    
        if sample['doc_index'] not in doc_context:
            negative_samples.append(sample['pivot_embeddings'][0])
    
    return negative_samples
    

def shuffle_data(x, c, y):
    shuffled = list(zip(x, c, y))
    random.shuffle(shuffled)

    x, c, y = zip(*shuffled)

    return x, c, y

In [62]:
target, embeddings, context, titles = load_data('/Users/julianbrendl/Projects/bachelor-thesis/word2doc/data/word2doc/1-wpp.npy')

Load /Users/julianbrendl/Projects/bachelor-thesis/word2doc/data/word2doc/1-wpp.npy


In [63]:
embeddings, context, target = shuffle_data(embeddings, context, target)

In [9]:
labels[0]

3987731

In [9]:
n_docs = 5316192
n_embedding = 300
n_sampled = 100

train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.float32, shape=(1, 4096), name='inputs')
    labels = tf.placeholder(tf.int64, shape=(1, 1) , name='labels')
    
    linear_w = tf.Variable(tf.truncated_normal([4096, 300]))
    linear_b = tf.Variable(tf.zeros([300]))

    linear_layer = tf.matmul(inputs, linear_w) + linear_b
    
    softmax_w = tf.Variable(tf.truncated_normal((n_docs, n_embedding))) # create softmax weight matrix here
    softmax_b = tf.Variable(tf.zeros(n_docs), name="softmax_bias") # create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(
        weights=softmax_w,
        biases=softmax_b,
        labels=labels,
        inputs=linear_layer,
        num_sampled=n_sampled,
        num_classes=n_docs)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [6]:
with train_graph.as_default():
    # From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size // 2))
    valid_examples = np.append(valid_examples, random.sample(range(1000, 1000 + valid_window), valid_size // 2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(linear_layer), 1, keep_dims=True))
    normalized_embedding = h1_dense / norm
    similarity = tf.matmul(normalized_embedding, tf.transpose(normalized_embedding))

In [75]:
def get_batches(embeddings, target):
    data = list(zip(embeddings, target))

    data_split = np.array_split(data, int(len(embeddings) / 8))

    for d in data_split:
        x, y = zip(*d)
        yield x, y

In [77]:
a = get_batch(embeddings, target)

In [92]:
for b in a:
    print(len(b))
    break

2


In [90]:
data = list(zip(embeddings, target))
data_split = np.array_split(data, int(len(embeddings) / 8))

In [70]:
a, b= zip(*data_split[0])

In [93]:
len(a)

TypeError: object of type 'generator' has no len()

In [10]:
epochs = 10
batch_size = 8
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        start = time.time()
        batches = get_batch(embeddings, target)
        for batch in batches:
            feed = {inputs: batch[0], labels: batch[1]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            print(loss)
                        
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
#             if iteration % 1000 == 0:
#                 ## From Thushan Ganegedara's implementation
#                 # note that this is expensive (~20% slowdown if computed every 500 steps)
#                 sim = similarity.eval()
#                 for i in range(valid_size):
#                     valid_word = int_to_vocab[valid_examples[i]]
#                     top_k = 8 # number of nearest neighbors
#                     nearest = (-sim[i, :]).argsort()[1:top_k+1]
#                     log = 'Nearest to %s:' % valid_word
#                     for k in range(top_k):
#                         close_word = int_to_vocab[nearest[k]]
#                         log = '%s %s,' % (log, close_word)
#                     print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

170.956039429
274.266960144
367.176185608
485.921287537


KeyboardInterrupt: 